<a href="https://colab.research.google.com/github/msc-acse/acse-9-independent-research-project-Garethlomax/blob/tests/module_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

imports

In [32]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision.transforms as transforms
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import random

import matplotlib.pyplot as plt
import h5py


torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

In [0]:
# %cd /content/drive/My \Drive/masters_project/
import clstm as cl# imports all

# from datasets import *

In [36]:
cl.train_main()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-36-e752fa51c29b>", line 1, in <module>
    cl.train_main()
AttributeError: module 'clstm' has no attribute 'train_main'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'AttributeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/dist-packages/IPyth

AttributeError: ignored

# cuda imports

In [0]:

device = 'cpu'
if torch.cuda.device_count() > 0 and torch.cuda.is_available():
    print("Cuda installed! Running on GPU!")
    print("GPUs:", torch.cuda.device_count())
    device = 'cuda'
else:
    print("No GPU available!")
    
    
import random
def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = True  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = True

    return True
  
set_seed(42)




# datasets

## image save

In [0]:
def test_image_save(model, train_loader, name, sample = 7, threshold = 0.5):
    model.eval()
    # calculate x and prediction 
    for a, b in train_loader:
        # a in input, b is truth
        break # train loader cannot be indexed
        
        
    with torch.no_grad():
        x = model(a.cuda())

    x = x.cpu()
    
    
    
    
    
#     print(x[sample][0][0])
    fig, axes = plt.subplots(1,2)
    print(x.shape)
    print(b.shape)
    axes[0].imshow(x[sample][0][0])
    axes[1].imshow(b[sample])
    
    axes[1].set_title("truth")
    axes[0].set_title("Prediction")
    fig.suptitle("Prediction of:" + name)
    fig.savefig(name + "sample"+ str(sample) + "comparison.pdf")
#     print(b[7])
#     print(x[7][0][0])
    plt.figure()
    x[sample][0][0][threshold > x[sample][0][0]] = 0
    plt.imshow(x[sample][0][0])
    fig, axes = plt.subplots(10,1,figsize=(32,32))
    for i in range(10):
        axes[i].imshow(a[sample][i][0])

In [0]:
def batch_loss_histogram(model, train_loader, loss_func):
    
    model.eval()
    # calculate x and prediction 
    for a, b in train_loader:
        # a in input, b is truth
        break # train loader cannot be indexed
        
        
    with torch.no_grad():
        x = model(a.cuda())
    
    
        x = x.cpu()
#     print(x.shape)
    # now over each one in x - we do
        #loss_func = nn.BCEWithLogitsLoss()
        loss = []
        for i in range(len(x)):
            loss.append(loss_func(x[i,:,0],b[i:i+1]).item())
        
    
    
    
    
    return loss


## wrapper

# model

In [0]:
structure = np.array([[12,24,0,0,0],[0,24,12,6,5]])

test_model = LSTMencdec_onestep(structure, 1, kernel_size = 5).to(device)

## code loading

In [0]:
%cd /content/drive/My Drive/masters_project/data/

avg = np.load("min_event_25_avg.npy")
std = np.load("min_event_25_std.npy")
# changed below
apbln = [0,1,0,0,1] # think this is correct
index_map = np.arange(0,52109,1)
train, valid = initialise_dataset_HDF5_full('data_min_events_25.hdf5', valid_frac = 0.1, dataset_length = 52109,avg = avg, std = std, application_boolean=apbln)

In [0]:
train_loader = DataLoader(valid, batch_size = 2000, shuffle = False) # implement moving MNIST data input


In [0]:
name = "bce_w_3"

In [0]:
test_model = nn.DataParallel(LSTMencdec_onestep(structure, 5, kernel_size = 3)).to(device) # added data parrallel

test_model.load_state_dict(torch.load(name + ".pth"))
test_model.eval()


In [0]:
test_image_save(test_model, train_loader, name + "comparison", sample = 1699, threshold = 0.76)
